## Description
In this Notebook the gensim/fasttext modell is used and trained. By the help of genism its possible to extend an existing model. But similar to the test from fasttext the model was overfitted afterwards.

#### Result
Preprocessing (Stemming eg.) worked good for the dataset. But the data seems not to be good for fine-tuning an existing modell. Afterwards the model was always overfitted, even after changing the training settings (different training rate etc.). Other approaches should be figured out.

The code was created with the assistance of ChatGPT-4.

In [ ]:
from gensim.models import FastText

model = FastText.load_fasttext_format('../01_Fasttext/data/cc.de.300.bin')

C:\Users\haabs\AppData\Local\Temp\ipykernel_40852\660162028.py:3: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  model = FastText.load_fasttext_format('../01_Fasttext/data/cc.de.300.bin')


In [ ]:
# Example: Similar words to "Auto" before training
similar_words_before = model.wv.most_similar("Mobilität", topn=10)
print("Vor dem Training:")
for word, similarity in similar_words_before:
    print(f"{word}: {similarity}")


Vor dem Training:
Mobiliät: 0.7578666806221008
Mobiltät: 0.7152034044265747
Automobilität: 0.6963497400283813
Alltagsmobilität: 0.6879281997680664
mobilität: 0.6839116215705872
Mobilitätskultur: 0.6754208207130432
Mobilitätsformen: 0.6675264835357666
Mitarbeitermobilität: 0.6643045544624329
Mobilitäten: 0.6508032083511353
Immobilität: 0.6439169049263


In [ ]:
import pandas as pd
# Example of custom sentences for further training of the model
df = pd.read_csv("data/dataset_descriptions_preprocessed.txt")  # Replace "deine_datei.csv" with the actual file path

# Convert the sentences from the columns into a list format for FastText
sentences = []
for col in df.columns:
    # Each cell in the column is treated as a list of words (tokens)
    for sentence in df[col].dropna():  # .dropna() removes empty cells
        sentences.append(sentence.split())  # Split the sentence into words and add it to the list

# Test: Are your words correctly processed?
for sentence in sentences[:10]:  # Show the first 10 tokenized sentences
    print(sentence)

['bern', 'ist', 'grün', 'und', 'vernetzt', 'ist', 'eine', 'illustrationskarte', 'aus', 'dem', 'räumlichen', 'stadtentwicklungskonzept', 'der', 'stadt', 'bern']
['bern', 'lebt', 'in', 'quartieren', 'ist', 'eine', 'illustrationskarte', 'aus', 'dem', 'räumlichen', 'stadtentwicklungskonzept', 'der', 'stadt', 'bern']
['bern', 'wächst', 'dynamisch', 'ist', 'eine', 'illustrationskarte', 'aus', 'dem', 'räumlichen', 'stadtentwicklungskonzept', 'der', 'stadt', 'bern']
['mountainbikeland', 'schweiz', 'von', 'schweizmobil', 'umfasst', 'die', 'nationalen', 'regionalen', 'und', 'lokalen', 'mountainbikerouten', 'der', 'schweiz', 'sowie', 'des', 'fürstentums', 'liechtenstein', 'dieser', 'datensatz', 'wird', 'gemäss', 'geoinformationsverordnung', 'als', 'teil', 'des', 'geobasisdatensatz', 'velowegnetze', 'publiziert', 'er', 'wird', 'in', 'zusammenarbeit', 'mit', 'dem', 'bundesamt', 'für', 'strassen', 'astra', 'schweizmobil', 'und', 'den', 'kantonen', 'erarbeitet']
['skatingland', 'schweiz', 'von', 'sch

In [ ]:
# Update the model with new vocabulary and sentence structures
model.build_vocab(sentences, update=True)  # Update vocabulary

model.alpha = 0.05  # Adjust initial learning rate
model.min_alpha = 0.05  # Set minimum learning rate
model.train(sentences, total_examples=len(sentences), epochs=10)

(1511321, 5244420)

In [ ]:
# Example: Similar words to "Auto" after training
similar_words_after = model.wv.most_similar("Mobilität", topn=10)
print("\nNach dem Training:")
for word, similarity in similar_words_after:
    print(f"{word}: {similarity}")



Nach dem Training:
eMobilität: 0.9988993406295776
e-Mobilität: 0.9987004995346069
GaseMobilität: 0.9982609152793884
Nobilität: 0.9979896545410156
E-Mobilität: 0.9979241490364075
WohnenMobilität: 0.994684100151062
bilität: 0.9895988702774048
Flexbilität: 0.9885423183441162
Debilität: 0.986996591091156
PflegeMobilität: 0.9811707139015198


In [ ]:
# safe trained model
model.save("data/erweitertes_fasttext_model.bin")
 